# SHapley Additive exPlanations (SHAP)

## Library Imports 

In [ ]:
import os
import pickle

import pandas as pd
import numpy as np
import shap
from matplotlib import pyplot as plt

In [ ]:
RAND_STATE = 0

## Importing the Train and Test Sets

In [ ]:
dataset_folder = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/datasets"

In [ ]:
X_train = pd.read_csv(os.path.join(dataset_folder, "obesity_X_train.csv"), index_col=0)

In [ ]:
X_test = pd.read_csv(os.path.join(dataset_folder, "obesity_X_test.csv"), index_col=0)

As with the rest of the stages, we make a variant of the sets without the height and weight columns for comparison:

In [ ]:
X_train_no_hw = X_train.drop(["Height", "Weight"], axis=1)

In [ ]:
X_test_no_hw = X_test.drop(["Height", "Weight"], axis=1)

For processing the SHAP values for one-hot encoded features, we will also need the unencoded sets:

In [ ]:
X_train_unencoded = pd.read_csv(os.path.join(dataset_folder, "obesity_X_train_unencoded.csv"), index_col=0)
X_train_unencoded

In [ ]:
X_test_unencoded = pd.read_csv(os.path.join(dataset_folder, "obesity_X_test_unencoded.csv"), index_col=0)
X_test_unencoded

## Importing the Random Forest Classifiers
We import the random forest classifiers that we trained previously. SHAP values will be calculated for these models to explain their predictions.

In [ ]:
def import_model(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/models/{filename}"
    with open(file_path, 'rb') as file: 
        model = pickle.load(file)
    print(f"Model imported from {file_path}")
    return model

In [ ]:
rand_forest = import_model("rand_forest.pkl")

In [ ]:
rand_forest_no_hw = import_model("rand_forest_no_hw.pkl")

## Importing Encoders
We import the label encoder for the target feature so that we can encode the original values for indexing purposes.

In [ ]:
def import_encoder(filename):
    file_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/encoders/{filename}"
    with open(file_path, 'rb') as file: 
        encoder = pickle.load(file)
    print(f"Encoder imported from {file_path}")
    return encoder

In [ ]:
target_le = import_encoder("target_le.pkl")

In [ ]:
target_class_label_d = {cls: idx for idx, cls in enumerate(target_le.classes_)}
target_class_label_d

We also import the one-hot encoder, which is needed when summing the SHAP values for one-hot encoded features:

In [ ]:
nominal_ohe = import_encoder("nominal_ohe.pkl")

## Explanations

### Calculating the SHAP Values

First, we create the explainers and calculate the SHAP values:

In [ ]:
explainer = shap.TreeExplainer(rand_forest)
shap_values = explainer(X_test)

In [ ]:
explainer_no_hw = shap.TreeExplainer(rand_forest_no_hw)
shap_values_no_hw = explainer_no_hw(X_test_no_hw)

### Undoing the One-Hot Encoding
However, because our nominal features are one-hot encoded, their SHAP values have been "separated". We need to group these one-hot encoded features back together. For reference, here is a list of all columns:

In [ ]:
X_test.columns

#### Summing Shape Values
We first need to calculate the SHAP values for each nominal feature, which is done by taking the sum of the SHAP values for each feature's corresponding set of one-hot encoded features. We begin by determining the number of output one-hot encoded features for each nominal feature:

In [ ]:
n_ohe_feats: dict[str, int] = {
    feat_name: (len(categories) if drop_idx is None else len(categories) - 1)
    for feat_name, categories, drop_idx
    in zip(nominal_ohe.feature_names_in_, nominal_ohe.categories_, nominal_ohe.drop_idx_)
}
n_ohe_feats

Our one-hot encoded dataset has the one-hot encoded features after all numerical features.
Hence, the number of numerical features will tell us the start index of the first one-hot encoded feature:

In [ ]:
numerical_feature_count = 8
numerical_feature_count_no_hw = 6

To calculate the sum, we first group the SHAP value columns by their original nominal features.
Then, we sum the SHAP values for each group column-wise.

In [ ]:
# Split the SHAP values for the one-hot encoded features.
# Each entry in split contains the SHAP values for the multiple one-hot encoded features of the original categorical feature.
values_split = np.split(shap_values.values[:, numerical_feature_count:, :], np.cumsum(list(n_ohe_feats.values())[:-1]), axis=1)

# Sum the SHAP values for each group.
values_summed = np.array([vals.sum(axis=1) for vals in values_split])

# We need to swap the first two axes since the first axis should index the instances and the second axis should index the features.
unohe_values = np.swapaxes(values_summed, 0, 1)

As a sanity check, we check the shape. The first axis's value should be the same as the number of instances, the second axis's value should be the number of nominal features and the third axis's value should be `2` since we only have two different categories for the target feature.

In [ ]:
unohe_values.shape

Finally, we concatenate back the numerical features and double check the shape again:

In [ ]:
new_values = np.concatenate((shap_values.values[:, :numerical_feature_count, :], unohe_values), axis=1)

In [ ]:
# 2nd axis should be 16, the number of original input features.
new_values.shape

At last, we can replace the old SHAP values:

In [ ]:
shap_values.values = new_values

#### Fixing the Data Values

Unfortunately, we are not done. We've replaced the SHAP values, but the data values are still one-hot encoded!

In [ ]:
shap_values.data[0]

In [ ]:
unohe_data = nominal_ohe.inverse_transform(shap_values.data[:, numerical_feature_count:])
unohe_data

In [ ]:
new_data = np.concatenate((shap_values.data[:, :numerical_feature_count], unohe_data), axis=1)
new_data.shape

In [ ]:
shap_values.data = new_data

#### Fixing Feature Names
Lastly, we need to update the feature names since the old feature names includes the one-hot encoded features. This is straightforward:

In [ ]:
new_feature_names = X_test.columns[:numerical_feature_count].to_list() + list(n_ohe_feats.keys())
new_feature_names

In [ ]:
shap_values.feature_names = new_feature_names

### With Height and Weight
Now, we can plot beeswarm plots for the model trained with the height and weight using the SHAP values.
For the obese class:

In [ ]:
shap.plots.beeswarm(shap_values[:, :, target_class_label_d["Yes"]], max_display=X_test.shape[1])

For the non-obese class:

In [ ]:
shap.plots.beeswarm(shap_values[:, :, target_class_label_d["No"]], max_display=X_test.shape[1])

### Without Height and Weight
Similarly, we plot beeswarm plots for the model trained without the height and weight.
For the obese class:

In [ ]:
shap.plots.beeswarm(grouped_shap_values_no_hw[:, :, target_class_label_d["Yes"]], max_display=X_test_no_hw.shape[1])

In [ ]:
shap.plots.scatter(grouped_shap_values_no_hw[:, "CAEC", 0], color=grouped_shap_values_no_hw[:, :, 0])

In [ ]:
def boxplot_categories(shap_values, feature: str, target_class: int):
    values = shap_values[:, feature, target_class].values
    data = shap_values[:, feature, target_class].data
    categories = np.unique(data).astype("int")
    
    groups = []
    for c in categories:
        relevant_values = values[data == c]
        groups.append(relevant_values)
    
    labels = categories
    
    plt.figure(figsize=(8, 5))
    plt.boxplot(groups, tick_labels=labels)
    plt.ylabel('SHAP Values', size=15)
    plt.xlabel('Obesity', size=15);

In [ ]:
    # "CAEC_Frequently": "CAEC",
    # "CAEC_Sometimes": "CAEC",
    # "CAEC_no": "CAEC",

np.unique(grouped_shap_values_no_hw[:, "CAEC", 0].data)

In [ ]:
boxplot_categories(grouped_shap_values_no_hw, "CAEC", 0)

For the non-obese class:

In [ ]:
shap.plots.beeswarm(grouped_shap_values_no_hw[:, :, target_class_label_d["No"]], max_display=X_test_no_hw.shape[1])